In [1]:
import numpy as np
import cv2
import keras
import glob
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from keras.utils import to_categorical
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout,  BatchNormalization , Activation
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from matplotlib import pyplot as plt
from PIL import Image
from tf_keras_vis.utils import print_gpus


Using TensorFlow backend.


In [4]:
#npy for train data
train = []
train_labels = []
files = glob.glob ("output/train/0/*.jpg") # your image path
for myFile in files:
    image = cv2.imread (myFile)
    train.append (image)
    train_labels.append([0])
files = glob.glob ("output/train/1/*.jpg")
for myFile in files:
    image = cv2.imread (myFile)
    train.append (image)
    train_labels.append([1])
files = glob.glob ("output/train/2/*.jpg")
for myFile in files:
    image = cv2.imread (myFile)
    train.append (image)
    train_labels.append([2])

test = []
test_labels = []
files = glob.glob ("output/test/0/*.jpg") # your image path
for myFile in files:
    image = cv2.imread (myFile)
    test.append (image)
    test_labels.append([0])
files = glob.glob ("output/test/1/*.jpg")
for myFile in files:
    image = cv2.imread (myFile)
    test.append (image)
    test_labels.append([1])
files = glob.glob ("output/test/2/*.jpg")
for myFile in files:
    image = cv2.imread (myFile)
    test.append (image)
    test_labels.append([2])

val = []
val_labels = []
files = glob.glob ("output/val/0/*.jpg") # your image path
for myFile in files:
    image = cv2.imread (myFile)
    val.append (image)
    val_labels.append([0])
files = glob.glob ("output/val/1/*.jpg")
for myFile in files:
    image = cv2.imread (myFile)
    val.append (image)
    val_labels.append([1])
files = glob.glob ("output/val/2/*.jpg")
for myFile in files:
    image = cv2.imread (myFile)
    val.append (image)
    val_labels.append([2])

In [5]:
train = np.array(train) #as mnist
train_labels = np.array(train_labels) #as mnist
test = np.array(test) #as mnist
test_labels = np.array(test_labels) #as mnist
val = np.array(val) #as mnist
val_labels = np.array(val_labels) #as mnist
np.save('trainx.npy', train)
np.save('trainy.npy',train_labels)
np.save('testx.npy', test)
np.save('testy.npy',test_labels)
np.save('valx.npy', val)
np.save('valy.npy',val_labels)

In [2]:
train_x = np.load('trainx.npy')
train_y = np.load('trainy.npy')
val_x = np.load('valx.npy')
val_y = np.load('valy.npy')

In [3]:
IMG_SIZE = 300
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

In [4]:
# Train, evaluate and save a Transfer Learning Model of ResNet50, VGG19 and DenseNet169

base_model = tf.keras.applications.ResNet50(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
base_model.trainable = False
x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(4096, activation='relu')(x)
x = tf.keras.layers.Dense(3, activation='softmax')(x)
model_3 = tf.keras.models.Model(inputs=base_model.input, outputs=x)
model_3.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [5]:
model_3.fit(train_x, train_y, epochs=12, batch_size=64, validation_data=(val_x, val_y))

Epoch 1/12
34/34 [==============================] - 152s 4s/step - loss: 0.9370 - accuracy: 0.8303 - val_loss: 0.2154 - val_accuracy: 0.9351
Epoch 2/12
34/34 [==============================] - 147s 4s/step - loss: 0.1379 - accuracy: 0.9508 - val_loss: 0.0796 - val_accuracy: 0.9684
Epoch 3/12
34/34 [==============================] - 148s 4s/step - loss: 0.0890 - accuracy: 0.9691 - val_loss: 0.1269 - val_accuracy: 0.9534
Epoch 4/12
34/34 [==============================] - 145s 4s/step - loss: 0.0951 - accuracy: 0.9644 - val_loss: 0.0686 - val_accuracy: 0.9734
Epoch 5/12
34/34 [==============================] - 260s 8s/step - loss: 0.0708 - accuracy: 0.9761 - val_loss: 0.1183 - val_accuracy: 0.9617
Epoch 6/12
34/34 [==============================] - 164s 5s/step - loss: 0.0636 - accuracy: 0.9756 - val_loss: 0.0952 - val_accuracy: 0.9684
Epoch 7/12
34/34 [==============================] - 297s 9s/step - loss: 0.0643 - accuracy: 0.9752 - val_loss: 0.1280 - val_accuracy: 0.9517
Epoch 8/12
34

In [6]:
test_x = np.load('testx.npy')
test_y = np.load('testy.npy')

In [7]:
results = model_3.evaluate(test_x, test_y)

19/19 [==============================] - 70s 4s/step - loss: 0.0604 - accuracy: 0.9801


In [8]:
model_3.save('corona_resnet.h5') 